In [10]:
from bs4 import BeautifulSoup as bs
from bs4 import SoupStrainer
from selenium import webdriver
import time, re
import requests
from itertools import repeat
import pandas as pd
import numpy as np
import unicodedata2
from tqdm import tqdm 

In [11]:
link_list = []
unique_id = []
speech_type = []

In [12]:
types_de_speech = ['Interview']
for i in tqdm(range(0,40)):
    link = 'https://www.vie-publique.fr/discours' +'?page='+ str(i)
    page_resp=requests.get(link)
    content= bs(page_resp.content,'html.parser',parse_only=SoupStrainer('main'))
    for link in content.find_all("a", class_='link-multiple'):
        if link.find('span').text.startswith(tuple(types_de_speech)):
            link_list.append('https://www.vie-publique.fr'+ link.get('href'))
            unique_id.append(link.get('href').split('/')[2].split('-')[0])
            speech_type.append(link.find('span').text.split(' ')[0])




  0%|          | 0/40 [00:00<?, ?it/s]


  2%|▎         | 1/40 [00:00<00:09,  4.24it/s]


  5%|▌         | 2/40 [00:00<00:09,  3.95it/s]


  8%|▊         | 3/40 [00:00<00:09,  4.11it/s]


 10%|█         | 4/40 [00:01<00:10,  3.56it/s]


 12%|█▎        | 5/40 [00:01<00:10,  3.34it/s]


 15%|█▌        | 6/40 [00:01<00:09,  3.52it/s]


 18%|█▊        | 7/40 [00:02<00:09,  3.34it/s]


 20%|██        | 8/40 [00:02<00:09,  3.33it/s]


 22%|██▎       | 9/40 [00:02<00:09,  3.14it/s]


 25%|██▌       | 10/40 [00:02<00:08,  3.40it/s]


 28%|██▊       | 11/40 [00:03<00:08,  3.30it/s]


 30%|███       | 12/40 [00:03<00:08,  3.32it/s]


 32%|███▎      | 13/40 [00:03<00:07,  3.53it/s]


 35%|███▌      | 14/40 [00:04<00:06,  3.81it/s]


 38%|███▊      | 15/40 [00:04<00:07,  3.55it/s]


 40%|████      | 16/40 [00:04<00:07,  3.15it/s]


 42%|████▎     | 17/40 [00:05<00:07,  2.98it/s]


 45%|████▌     | 18/40 [00:05<00:07,  3.11it/s]


 48%|████▊     | 19/40 [00:05<00:07,  2.94it/s]


 50%|█████     

Essai sur les premières pages 

In [13]:
LINKS=pd.DataFrame({'unique_id':unique_id,'speech_type':speech_type,'link':link_list})
LINKS=LINKS.drop_duplicates(subset=['unique_id'],ignore_index=True)
LINKS.head()

,unique_id,speech_type,link
0,274025,Interview,https://www.vie-publique.fr/discours/274025-em...
1,273925,Interview,https://www.vie-publique.fr/discours/273925-br...
2,273923,Interview,https://www.vie-publique.fr/discours/273923-ju...
3,273865,Interview,https://www.vie-publique.fr/discours/273865-si...
4,273919,Interview,https://www.vie-publique.fr/discours/273919-je...


In [14]:
LINKS

,unique_id,speech_type,link
0,274025,Interview,https://www.vie-publique.fr/discours/274025-em...
1,273925,Interview,https://www.vie-publique.fr/discours/273925-br...
2,273923,Interview,https://www.vie-publique.fr/discours/273923-ju...
3,273865,Interview,https://www.vie-publique.fr/discours/273865-si...
4,273919,Interview,https://www.vie-publique.fr/discours/273919-je...
...,...,...,...
143,272850,Interview,https://www.vie-publique.fr/discours/272850-mu...
144,272790,Interview,https://www.vie-publique.fr/discours/272790-ag...
145,272789,Interview,https://www.vie-publique.fr/discours/272789-ge...
146,272870,Interview,https://www.vie-publique.fr/discours/272870-em...


In [15]:
length = '0-800-2902020'
LINKS.to_csv('DATA_' +length+'.csv')

Quelles infos extra prendre ? 
- Intervenant (PB quand plusieurs si non journaliste ? ) 
- Fonction (OK mais possibles erreurs à checker ! ) 
- Titre 
- date # mettre as date 
- sujet principal (about) # pas encore fait
- Thématique 
- publications dans d'autres médias ? # pas encore fait
- Interview : mettre radio ou journal considéré
- discours ( à checker)

Ex : pb avec https://www.vie-publique.fr/discours/272865-ministere-de-leurope-et-des-affaires-etrangeres-20012020-ormuz 

In [16]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
#chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument("--disable-extensions")

# Selenium

In [17]:
prenom=[]
nom=[]
discours=[]
merge_id=[]


In [18]:
# Premier code d'extraction, deuxième code après le premier round pour ceux qui ont pas marché. 
#print('DATA_' +length)
from selenium.common.exceptions import NoSuchElementException
d = webdriver.Chrome(options=chrome_options)
for j in tqdm(range(len(LINKS.link))):
    try:
        d.get(LINKS.link[j])
    except:
        pass
    
    merge_id.append(LINKS.unique_id[j])
    try:
        prenom.append(d.find_element_by_class_name("line-intervenant").text.split(' - ')[0].split(' ')[0])
    except: 
        prenom.append(np.nan)
    try:
        nom.append(' '.join(d.find_element_by_class_name("line-intervenant").text.split('\n')[0].split(' - ')[0].split(' ')[1:]))
    except:
        nom.append(np.nan)
    try: 
        para=d.find_element_by_xpath("//span[contains(@class, 'texte-integral')]").find_element_by_xpath("//span[contains(@class, 'texte-integral')]").text.split('\n')
        para = [x for x in para if x != '']
        discours_s= []
        # A rassembler les if en un seul quand même code avec liste et isin 
        for i in range(len(para)-1):
            nomprenom=d.find_element_by_class_name("line-intervenant").text.split(' - ')[0].upper()
            if para[i].startswith(nomprenom):
                if not para[i].replace(nomprenom,'')==' ':
                    discours_s.append(para[i].replace(nomprenom,''))
                if not para[i+1].replace(nomprenom,'')==' ':
                    discours_s.append(para[i+1].replace(nomprenom,''))
                if len(discours_s)==0:
                    for i in range(len(para)-1):
                        if para[i].startswith('R -'):
                            discours_s.append(para[i].replace('R -',''))
                if len(discours_s)==0:
                    prenom_nom=d.find_element_by_class_name("line-intervenant").text.split(' - ')[0].split(' ')[0] + ' '+ ' '.join(d.find_element_by_class_name("line-intervenant").text.split('\n')[0].split(' - ')[0].split(' ')[1:]) +' :'
                    for i in range(len(para)-1):
                        if para[i].startswith(prenom_nom):
                            discours_s.append(para[i].strip(prenom_nom))
                if len(discours_s)==0:
                    if para[0].startswith('?'):
                        for i in range(len(para)-1):
                            if not para[i].startswith('?'):
                                discours_s.append(para[i])              
    except:
        discours.append(np.nan)
    #time.sleep(1)
    #print(j)
d.close()
print(j)

# J'étais obligée les try except pour chaque truc ca bug à chaque fois sinon ... Autre méthode ? 
                                      
# Plus je pense que la ca fonctionne jusquà la page 1000, j'ai un soucis de nom aussi 
                                      




  0%|          | 0/148 [00:00<?, ?it/s]


  1%|          | 1/148 [00:03<08:25,  3.44s/it]


  1%|▏         | 2/148 [00:07<08:57,  3.68s/it]


  2%|▏         | 3/148 [00:09<07:24,  3.07s/it]


  3%|▎         | 4/148 [00:10<06:01,  2.51s/it]


  3%|▎         | 5/148 [00:12<05:47,  2.43s/it]


  4%|▍         | 6/148 [00:16<06:19,  2.67s/it]


  5%|▍         | 7/148 [00:19<06:39,  2.83s/it]


  5%|▌         | 8/148 [00:22<06:45,  2.89s/it]


  6%|▌         | 9/148 [00:26<07:46,  3.36s/it]


  7%|▋         | 10/148 [00:27<06:16,  2.73s/it]


  7%|▋         | 11/148 [00:29<05:31,  2.42s/it]


  8%|▊         | 12/148 [00:32<05:47,  2.55s/it]


  9%|▉         | 13/148 [00:34<05:28,  2.43s/it]


  9%|▉         | 14/148 [00:36<04:55,  2.21s/it]


 10%|█         | 15/148 [00:41<06:38,  3.00s/it]


 11%|█         | 16/148 [00:43<06:10,  2.80s/it]


 11%|█▏        | 17/148 [00:45<05:47,  2.65s/it]


 12%|█▏        | 18/148 [00:47<05:22,  2.48s/it]


 13%|█▎        | 19/148 [00:50<05:06,  2.38s/i

 92%|█████████▏| 136/148 [07:34<01:01,  5.15s/it]


 93%|█████████▎| 137/148 [07:37<00:49,  4.47s/it]


 93%|█████████▎| 138/148 [07:43<00:48,  4.81s/it]


 94%|█████████▍| 139/148 [07:46<00:40,  4.50s/it]


 95%|█████████▍| 140/148 [07:49<00:32,  4.02s/it]


 95%|█████████▌| 141/148 [07:55<00:31,  4.45s/it]


 96%|█████████▌| 142/148 [07:57<00:23,  3.91s/it]


 97%|█████████▋| 143/148 [08:00<00:17,  3.42s/it]


 97%|█████████▋| 144/148 [08:07<00:18,  4.70s/it]


 98%|█████████▊| 145/148 [08:11<00:13,  4.46s/it]


 99%|█████████▊| 146/148 [08:15<00:08,  4.15s/it]


 99%|█████████▉| 147/148 [08:18<00:03,  3.90s/it]


100%|██████████| 148/148 [08:24<00:00,  3.41s/it]


147


In [6]:
prenom=[]
nom=[]
discours=[]
merge_id=[]


In [ ]:
from selenium.common.exceptions import NoSuchElementException
d = webdriver.Chrome(options=chrome_options)
for j in tqdm(range(0,200)): # len(LINKS.link)
    try:
        d.get(LINKS.link[j])
    except:
        pass
    
    merge_id.append(LINKS.unique_id[j])
    try:
        prenom.append(d.find_element_by_class_name("line-intervenant").text.split(' - ')[0].split(' ')[0])
    except: 
        prenom.append(np.nan)
    try:
        nom.append(' '.join(d.find_element_by_class_name("line-intervenant").text.split('\n')[0].split(' - ')[0].split(' ')[1:]))
    except:
        nom.append(np.nan)
    try: 
        para=d.find_element_by_xpath("//span[contains(@class, 'texte-integral')]").find_element_by_xpath("//span[contains(@class, 'texte-integral')]").text.split('\n')
        para = [x for x in para if x != '']
        discours_s= []
        # A rassembler les if en un seul quand même code avec liste et isin 
        for i in range(len(para)-1):
            nomprenom=d.find_element_by_class_name("line-intervenant").text.split(' - ')[0].upper()
            if para[i].startswith(nomprenom):
                if not para[i].replace(nomprenom,'')==' ':
                    discours_s.append(para[i].replace(nomprenom,''))
                if not para[i+1].replace(nomprenom,'')==' ':
                    discours_s.append(para[i+1].replace(nomprenom,''))
                if len(discours_s)==0:
                    for i in range(len(para)-1):
                        if para[i].startswith('R -'):
                            discours_s.append(para[i].replace('R -',''))
                if len(discours_s)==0:
                    prenom_nom=d.find_element_by_class_name("line-intervenant").text.split(' - ')[0].split(' ')[0] + ' '+ ' '.join(d.find_element_by_class_name("line-intervenant").text.split('\n')[0].split(' - ')[0].split(' ')[1:]) +' :'
                    for i in range(len(para)-1):
                        if para[i].startswith(prenom_nom):
                            discours_s.append(para[i].strip(prenom_nom))
                if len(discours_s)==0:
                    if para[0].startswith('?'):
                        for i in range(len(para)-1):
                            if not para[i].startswith('?'):
                                discours_s.append(para[i])              
    except:
        discours.append(np.nan)
    #time.sleep(1)
    #print(j)
d.close()
print(j)




  0%|          | 0/200 [00:00<?, ?it/s]


  0%|          | 1/200 [00:03<10:35,  3.20s/it]


  1%|          | 2/200 [00:07<11:37,  3.52s/it]


  2%|▏         | 3/200 [00:08<09:15,  2.82s/it]


  2%|▏         | 4/200 [00:09<07:45,  2.37s/it]


  2%|▎         | 5/200 [00:11<07:10,  2.21s/it]


  3%|▎         | 6/200 [00:14<07:16,  2.25s/it]


  4%|▎         | 7/200 [00:17<08:05,  2.51s/it]


  4%|▍         | 8/200 [00:18<07:15,  2.27s/it]


  4%|▍         | 9/200 [00:21<07:49,  2.46s/it]


  5%|▌         | 10/200 [00:22<06:11,  1.95s/it]


  6%|▌         | 11/200 [00:23<05:31,  1.75s/it]


  6%|▌         | 12/200 [00:26<06:40,  2.13s/it]


  6%|▋         | 13/200 [00:28<06:13,  2.00s/it]


  7%|▋         | 14/200 [00:30<05:51,  1.89s/it]


  8%|▊         | 15/200 [00:34<07:33,  2.45s/it]


  8%|▊         | 16/200 [00:36<07:17,  2.38s/it]


  8%|▊         | 17/200 [00:37<06:09,  2.02s/it]


  9%|▉         | 18/200 [00:38<05:37,  1.86s/it]


 10%|▉         | 19/200 [00:40<05:32,  1.84s/i

 68%|██████▊   | 136/200 [08:07<04:26,  4.16s/it]


 68%|██████▊   | 137/200 [08:09<03:53,  3.70s/it]

In [ ]:
discours

# Beautiful SOup

In [21]:
discours 

[]

In [20]:
DISCOURS = pd.DataFrame({'merge_id':merge_id,
                         'discours':discours})

ValueError: arrays must all be same length

In [ ]:
DATA.to_csv('DATA.csv')
DISCOURS.to_csv('DISCOURS.csv')

In [113]:
DATA=pd.read_csv('DATA.csv')
DISCOURS=pd.read_csv('DISCOURS.csv')
MOTS_CLE=pd.read_csv('MOTS_CLE.csv')

In [114]:
len(DATA),len(DISCOURS),len(MOTS_CLE)

(7299, 7299, 7299)

In [115]:
for i in range(len(mots_cle)):
    if mots_cle[i] is np.nan:
        mots_cle[i]=''
    else:
        mots_cle[i]=mots_cle[i].replace(' - ','-')
        mots_cle[i]=mots_cle[i].replace("INSTITUTIONS DE L'ETAT","INSTITUTIONS-DE-L'ETAT")
        mots_cle[i]=mots_cle[i].replace(' EUROP','-EUROP')
        mots_cle[i]=mots_cle[i].replace('VIE EC','VIE-EC')
        mots_cle[i]=mots_cle[i].replace('ITIQUE ','ITIQUE-')
        mots_cle[i]=mots_cle[i].replace('ZONE ','ZONE-')
        mots_cle[i]=mots_cle[i].replace('RELATIONS ','RELATIONS-')
        mots_cle[i]=mots_cle[i].split(' ') 
# Faire une sorte de regroupement de mots cle avec l'algo vu en cours ? 


AttributeError: 'list' object has no attribute 'replace'

In [116]:
col= []
for i in range(len(max([x for x in mots_cle if x is not np.nan], key=len))):
    col.append('MC'+str(i+1))
col

['MC1', 'MC2', 'MC3', 'MC4', 'MC5', 'MC6', 'MC7', 'MC8', 'MC9', 'MC10']

In [117]:
MOTS_CLE=pd.DataFrame(data=mots_cle,columns=col)
MOTS_CLE.to_csv('MOTS_CLE.csv')

In [118]:
DATA=pd.concat([DATA,MOTS_CLE],axis=1)
DATA=DATA.merge(DISCOURS, left_on='unique_id',right_on='merge_id')
DATA.columns
DATA.head()

,Unnamed: 0_x,titre,link,speech_type,prenom,nom,date,theme,fonct,unique_id,...,MC4,MC5,MC6,MC7,MC8,MC9,MC10,Unnamed: 0_y,merge_id,discours
0,0,"Déclaration de Mme Brune Poirson, secrétaire d...",https://www.vie-publique.fr/discours/272458-br...,Déclaration,Brune,Poirson,2019-12-03T12:00:00Z,SOCIÉTÉ,Secrétaire d'Etat auprès du ministre de la tra...,272458,...,None,None,None,None,None,None,None,0,272458,"Monsieur le Député, Vous avez raison de pointe..."
1,1,"Déclaration de M. Jean-Michel Blanquer, minist...",https://www.vie-publique.fr/discours/272635-je...,Déclaration,Jean-Michel,Blanquer,2019-12-03T12:00:00Z,SOCIÉTÉ,Ministre de l'éducation nationale et de la jeu...,272635,...,None,None,None,None,None,None,None,1,272635,"Mesdames et messieurs les Professeurs, Le Gou..."
2,2,"Déclarations de M. Bruno Le Maire, ministre de...",https://www.vie-publique.fr/discours/272488-br...,Déclarations,Bruno,Le Maire,2019-12-03T12:00:00Z,SOCIÉTÉ,Ministre de l'économie et des finances Marlène...,272488,...,None,None,None,None,None,None,None,2,272488,NaN
3,3,Extraits d'un entretien de M. Jean-Baptiste Le...,https://www.vie-publique.fr/discours/272186-en...,Extraits,Jean-Baptiste,Lemoyne,2019-12-02T12:00:00Z,INTERNATIONAL,Secrétaire d'État auprès du ministre de l'Euro...,272186,...,None,None,None,None,None,None,None,3,272186,Je souhaite toute mon émotion et toute l'émot...
4,4,"Interview de M. Jean-Michel Blanquer, ministre...",https://www.vie-publique.fr/discours/272235-je...,Interview,Jean-Michel,Blanquer,2019-12-02T12:00:00Z,SOCIÉTÉ,Ministre de l'éducation nationale et de la jeu...,272235,...,None,None,None,None,None,None,None,4,272235,Bonjour Alba VENTURA. C'est toujours importa...


In [119]:
col=[col for col in DATA if col.startswith('MC')]

In [120]:
for C in col : 
    DATA[C]=DATA[C].replace(to_replace=[None], value=' ')

In [121]:
DATA=DATA[~DATA.prenom.isna()]

In [122]:
len(DATA)

7226

In [123]:
#DATA=DATA[~DATA.fonct.isna()]

In [124]:
# Pour les discours plus vieux, la fonction se trouve uniquement dans le titre 
#DATA['fonct'] = DATA.apply(
#    lambda row: row['titre'].split(',')[1] if np.isnan(row['fonct']) else row['fonct'],
#    axis=1

#)

In [125]:
len(DATA)

7226

In [126]:
DATA.dropna(subset=['discours'],inplace=True)

In [127]:
DATA=DATA[DATA.discours != '']
DATA.reset_index(inplace=True)
del DATA['index']
del DATA['level_0']

KeyError: 'level_0'

In [128]:
DATA.to_csv('Sample_51_800.csv')

In [129]:
len(DATA)

5737

In [130]:
#Faut voir avec le merge id si ca colle ... 

### Merge label prénoms

In [131]:
pd.read_csv('Sample_51_800.csv')

,Unnamed: 0,Unnamed: 0_x,titre,link,speech_type,prenom,nom,date,theme,fonct,...,MC4,MC5,MC6,MC7,MC8,MC9,MC10,Unnamed: 0_y,merge_id,discours
0,0,0,"Déclaration de Mme Brune Poirson, secrétaire d...",https://www.vie-publique.fr/discours/272458-br...,Déclaration,Brune,Poirson,2019-12-03T12:00:00Z,SOCIÉTÉ,Secrétaire d'Etat auprès du ministre de la tra...,...,,,,,,,,0,272458,"Monsieur le Député, Vous avez raison de pointe..."
1,1,1,"Déclaration de M. Jean-Michel Blanquer, minist...",https://www.vie-publique.fr/discours/272635-je...,Déclaration,Jean-Michel,Blanquer,2019-12-03T12:00:00Z,SOCIÉTÉ,Ministre de l'éducation nationale et de la jeu...,...,,,,,,,,1,272635,"Mesdames et messieurs les Professeurs, Le Gou..."
2,2,3,Extraits d'un entretien de M. Jean-Baptiste Le...,https://www.vie-publique.fr/discours/272186-en...,Extraits,Jean-Baptiste,Lemoyne,2019-12-02T12:00:00Z,INTERNATIONAL,Secrétaire d'État auprès du ministre de l'Euro...,...,,,,,,,,3,272186,Je souhaite toute mon émotion et toute l'émot...
3,3,4,"Interview de M. Jean-Michel Blanquer, ministre...",https://www.vie-publique.fr/discours/272235-je...,Interview,Jean-Michel,Blanquer,2019-12-02T12:00:00Z,SOCIÉTÉ,Ministre de l'éducation nationale et de la jeu...,...,,,,,,,,4,272235,Bonjour Alba VENTURA. C'est toujours importa...
4,4,5,"Déclaration de M. Emmanuel Macron, président d...",https://www.vie-publique.fr/discours/272165-em...,Déclaration,Emmanuel,Macron,2019-12-02T12:00:00Z,INTERNATIONAL,Président de la République,...,,,,,,,,5,272165,"Officiers, sous-officiers, soldats, marins, av..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5732,5732,7294,"Déclaration de M. Harlem Désir, secrétaire d'E...",https://www.vie-publique.fr/discours/200488-de...,Déclaration,Harlem,Désir,2016-09-01T12:00:00Z,INTERNATIONAL,NaN,...,,,,,,,,7294,200488,Madame la Présidente de la Commission des affa...
5733,5733,7295,"Déclaration de M. Jean-Marc Todeschini, secrét...",https://www.vie-publique.fr/discours/200416-de...,Déclaration,Jean-Marc,Todeschini,2016-09-01T12:00:00Z,SOCIÉTÉ,NaN,...,,,,,,,,7295,200416,"Monsieur le Préfet, Monsieur le Député-Préside..."
5734,5734,7296,"Déclaration de M. Manuel Valls, Premier minist...",https://www.vie-publique.fr/discours/200320-de...,Déclaration,Manuel,Valls,2016-09-01T12:00:00Z,INTERNATIONAL,NaN,...,,,,,,,,7296,200320,"Messieurs les ministres, Mesdames, messieurs l..."
5735,5735,7297,"Déclaration de M. Jean-Marc Todeschini, secrét...",https://www.vie-publique.fr/discours/200407-de...,Déclaration,Jean-Marc,Todeschini,2016-09-01T12:00:00Z,INTERNATIONAL,NaN,...,,,,,,,,7297,200407,"Monsieur le Préfet, Mesdames et messieurs les ..."


In [132]:
# On peut aussi aller chercher le madame monsieur ! 
prenoms=pd.read_csv(r'C:\Users\morga\Documents\GitHub\NLP_brouillon\nat2018.csv',sep=';')
prenoms=prenoms.groupby(['preusuel','sexe'],as_index=False).sum()
prenoms=prenoms.sort_values(['nombre'],ascending=True)
prenoms=prenoms.drop_duplicates(subset=['preusuel'],keep='last') # On prend le sexe qui apparait le plus 
prenoms=prenoms[['preusuel','sexe']]
DATA.prenom=[pre.upper() for pre in DATA.prenom.astype('string')]
DATA=DATA.merge(prenoms, left_on='prenom',right_on='preusuel')

In [135]:
DATA.to_csv('Sample_51_800.csv')

In [134]:
DATA.columns

Index(['Unnamed: 0_x', 'titre', 'link', 'speech_type', 'prenom', 'nom', 'date',
       'theme', 'fonct', 'unique_id', 'MC1', 'MC2', 'MC3', 'MC4', 'MC5', 'MC6',
       'MC7', 'MC8', 'MC9', 'MC10', 'Unnamed: 0_y', 'merge_id', 'discours',
       'preusuel', 'sexe'],
      dtype='object')

### Themes 

In [104]:
DATA=pd.read_csv('Sample_0_200.csv')

In [105]:
DATA=DATA.drop(columns='preusuel')

In [106]:
theme_clean=[]
for theme in DATA.theme:
    if len(theme.split('\n'))==0:
        theme_clean.append(theme)
    if len(theme.split('\n'))>0:
        theme_clean.append(theme.split('\n')[0])
    

AttributeError: 'float' object has no attribute 'split'

In [ ]:
DATA.theme=theme_clean

In [107]:
DATA

,Unnamed: 0,Unnamed: 0_x,titre,link,speech_type,prenom,nom,date,theme,fonct,...,MC5,MC6,MC7,MC8,MC9,MC10,Unnamed: 0_y,merge_id,discours,sexe
0,0,0,"Déclaration de Mme Brune Poirson, secrétaire d...",https://www.vie-publique.fr/discours/272458-br...,Déclaration,BRUNE,Poirson,2019-12-03T12:00:00Z,SOCIÉTÉ,Secrétaire d'Etat auprès du ministre de la tra...,...,,,,,,,0,272458,"Monsieur le Député, Vous avez raison de pointe...",2
1,1,17,"Interview de Mme Brune Poirson, secrétaire d'É...",https://www.vie-publique.fr/discours/272457-br...,Interview,BRUNE,Poirson,2019-11-28T12:00:00Z,SOCIÉTÉ,Secrétaire d'État auprès de la ministre de la ...,...,,,,,,,17,272457,Bonjour Jean-Jacques BOURDIN. J'ai plein de r...,2
2,2,38,"Interview de Mme Brune Poirson, secrétaire d'É...",https://www.vie-publique.fr/discours/272456-br...,Interview,BRUNE,Poirson,2019-11-26T12:00:00Z,SOCIÉTÉ,Secrétaire d'État auprès de la ministre de la ...,...,,,,,,,38,272456,Bonjour Caroline ROUX. Je pense qu'il y a un...,2
3,3,269,"Déclaration de Mme Brune Poirson, secrétaire d...",https://www.vie-publique.fr/discours/272455-br...,Déclaration,BRUNE,Poirson,2019-10-25T12:00:00Z,SOCIÉTÉ,Secrétaire d'État auprès du ministre de la tra...,...,,,,,,,269,272455,"M. le Ministre, Messieurs les Présidents, Mesd...",2
4,4,449,"Déclaration de de Mme Brune Poirson, secrétair...",https://www.vie-publique.fr/discours/271265-br...,Déclaration,BRUNE,Poirson,2019-10-03T12:00:00Z,SOCIÉTÉ,", Secrétaire d'État auprès de la ministre de l...",...,,,,,,,449,271265,Nous sommes deux ans après le discours de lanc...,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5581,5581,7263,"Interview de M. Manuel Valls, Premier ministre...",https://www.vie-publique.fr/discours/200319-in...,Interview,MANUEL,Valls,2016-09-06T12:00:00Z,INSTITUTIONS,NaN,...,,,,,,,7263,200319,"Bonjour Yves CALVI. Non, je ne le crois pas ...",1
5582,5582,7269,"Déclaration de M. Manuel Valls, Premier minist...",https://www.vie-publique.fr/discours/200358-de...,Déclaration,MANUEL,Valls,2016-09-06T12:00:00Z,INTERNATIONAL,NaN,...,,,,,,,7269,200358,En présence de Manuel VALLS Premier Ministre A...,1
5583,5583,7296,"Déclaration de M. Manuel Valls, Premier minist...",https://www.vie-publique.fr/discours/200320-de...,Déclaration,MANUEL,Valls,2016-09-01T12:00:00Z,INTERNATIONAL,NaN,...,,,,,,,7296,200320,"Messieurs les ministres, Mesdames, messieurs l...",1
5584,5584,6846,"Déclaration de Mme Hélène Geoffroy, secrétaire...",https://www.vie-publique.fr/discours/200893-de...,Déclaration,HÉLÈNE,Geoffroy,2016-10-19T12:00:00Z,SOCIÉTÉ,NaN,...,,,,,,,6846,200893,Mme la présidente. L'ordre du jour appelle le ...,2


In [108]:
DATA.to_csv('Sample_0_200.csv')